In [ ]:
"""
IMPORTS
"""

import pandas as pd
import numpy as np
import onnxruntime as rt


from utils.dataloader import DataLoader
from models.modelloader import ModelLoader
from utils.constants import protected_attributes, group_proxies


from tests.metrics import MetricsTester
from tests.mututation import MutationTester
from tests.metamorphic import MetamorphicTester
from tests.shapley import ShapleyTester

In [ ]:
"""
    INTERESTING LINKS:
        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937
        - https://github.com/interpretml/interpret-community
"""

In [1]:
"""
SET MODEL(S)
"""
MODEL_UNDER_TEST = './../../data/models/final_models/good_model.onnx'

In [ ]:
"""
LOAD DATASET
"""

dataloader = DataLoader()
X_train, y_train = dataloader.load_split('train')
print(f'{X_train.shape}, {y_train.shape}')
X_test, y_test = dataloader.load_split('test')
print(f'{X_test.shape}, {y_test.shape}')
X_data, y_data = dataloader.load_split('full')
print(f'{X_data.shape}, {y_data.shape}')

In [ ]:
"""
    TEST MODEL ON DATASET
"""

session = rt.InferenceSession(MODEL_UNDER_TEST)

y_pred = session.run(None, {'X': X_test.iloc[:, :-1].values.astype(np.float32)})[0]

In [ ]:
"""
FUNCTIONAL TESTING
"""

metrics = MetricsTester(protected_variables=protected_attributes, group_metrics=group_proxies)


In [ ]:
metrics.get_metrics_summary(X_test, y_true=y_test, y_pred=y_pred)

In [ ]:
"""
METAMORPHIC TESTING
"""

metamorphic_tester = MetamorphicTester(X=X_data, model_path=MODEL_UNDER_TEST)
metamorphic_tester.test()
metamorphic_tester.analyse_test()


In [ ]:
metamorphic_tester.global_average

In [ ]:
metamorphic_tester.column_avg

In [ ]:
"""
MUTATION TESTING
"""

params = {
    'data_shuffler': 1000,
    'data_remover': 0.25,
    'data_repetition': 1000, 
    'label_error': 1000,
    'feature_remover': 31,
}


mutator = MutationTester(y_pred_baseline=y_pred, df_train=X_train, df_test=X_test)
mutation_score = mutator.test_mutants(params)
print(f"Mutation score: {mutation_score}")

In [ ]:
"""
AUC
"""


In [ ]:
"""
COUNTERFACTUALS
"""